# The First Approach We Tried

#### Prerequisites

In [1]:
%%capture
!pip install lasio
!pip install welly

In [3]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
import welly
import collections
from welly import Well
from welly import Project
import csv
import lasio 
from tqdm.notebook import tqdm
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
import xgboost as xg
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
# from catboost import CatBoostRegressor
from sklearn.kernel_ridge import KernelRidge
from scipy import stats
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, Ridge, Lasso, BayesianRidge, SGDRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from scipy import stats
from sklearn.neural_network import MLPRegressor

import warnings
warnings.filterwarnings('ignore')

In [4]:
# CSV Containing Info about logs and units of logs  
!gdown --id 1-1bPAcTXXv2TqaQDMGHIgH0wvneQdVEZ
!unzip -qq /content/wellloginfo.zip -d 'wellloginfo'
os.remove("/content/wellloginfo.zip")

# Stacked file For both regions
!gdown --id 1--mnuePiANuTUuljKIj_W0lIP_tAJOXU
!unzip -qq /content/stacked-data-both-regions.zip -d 'stacked-data-both-regions'
os.remove("/content/stacked-data-both-regions.zip")

!gdown --id 1-43M2dHo-p7sBMQv0DK2RK8ZC1cNpVUC
!unzip -qq final-data.zip -d 'test-data'
os.remove("final-data.zip")

Downloading...
From: https://drive.google.com/uc?id=1-1bPAcTXXv2TqaQDMGHIgH0wvneQdVEZ
To: /content/wellloginfo.zip
100% 3.26k/3.26k [00:00<00:00, 3.55MB/s]
Downloading...
From: https://drive.google.com/uc?id=1--mnuePiANuTUuljKIj_W0lIP_tAJOXU
To: /content/stacked-data-both-regions.zip
120MB [00:02, 56.7MB/s]


#### Unit Values Selection and Prediction

In [6]:
dataF = pd.read_csv('/content/stacked-data-both-regions/data_stacked_T_ (1).csv')
dataF = dataF[dataF['DTSM'].notna()]
dataF.shape

(805987, 163)

In [10]:
las2 = lasio.read("/content/test-data/1684cc35f399_TGS.las")
test_data = las2.df()
test_data.head(5)

,GRR,ILD,ILM,SFLU,SPR,CALR,DTCO,NPHI,DPHI,PE,DRHO,CALD,RHOB
DEPT,,,,,,,,,,,,,
5994.0,82.4304,9.1583,8.6697,8.6378,-24.9283,9.4701,71.2017,NaN,NaN,NaN,NaN,NaN,NaN
5994.5,80.1221,9.1626,8.6134,9.4136,-26.6600,9.4440,70.2243,NaN,NaN,NaN,NaN,NaN,NaN
5995.0,77.1246,9.1152,8.4855,10.0892,-27.2562,9.4560,67.5032,NaN,NaN,NaN,NaN,NaN,NaN
5995.5,76.2724,8.9201,8.3596,10.0951,-28.3344,9.4822,60.7898,NaN,NaN,NaN,NaN,NaN,NaN
5996.0,77.7058,8.1558,8.1499,9.8005,-30.2277,9.5882,56.5389,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
cols_list = []
for i in test_data.columns:
    print(i)
    if i in dataF.columns:
        cols_list.append(i)


d1 = dataF[cols_list]

GRR
ILD
ILM
SFLU
SPR
CALR
DTCO
NPHI
DPHI
PE
DRHO
CALD
RHOB


In [16]:
unit = 'OHMM'
Wellloginfo = pd.read_csv("/content/wellloginfo/WellLogsInfo.csv")
unit_logs = pd.DataFrame()
for i in range(Wellloginfo.shape[0]):
    k = Wellloginfo.iloc[i, :]
    if k[2] == unit and k[0] in d1.columns:
        print(k[0])
        unit_logs[k[0]] = d1[k[0]]


print(unit_logs.info())
print(unit_logs.corr())

ILD
ILM
SFLU
<class 'pandas.core.frame.DataFrame'>
Int64Index: 805987 entries, 24224 to 1528014
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   ILD     219895 non-null  float64
 1   ILM     189581 non-null  float64
 2   SFLU    202497 non-null  float64
dtypes: float64(3)
memory usage: 24.6 MB
None
           ILD       ILM      SFLU
ILD   1.000000  0.652603  0.735762
ILM   0.652603  1.000000  0.667817
SFLU  0.735762  0.667817  1.000000


In [ ]:
cols_list = ['SPR','DEPT']
cols_list.append('DTSM')
unit_logs = unit_logs[cols_list]
cols_list.pop()
Xtrain = unit_logs.dropna(thresh = 3)
print(Xtrain.shape)

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(Xtrain[cols_list],Xtrain['DTSM'],test_size=0.3)
xgb_r = xg.XGBRegressor(booster = 'gbtree', objective='reg:squarederror', n_estimators=500, max_depth = 6)
xgb_r.fit(train_x,train_y)
pred = xgb_r.predict(test_x)
rmse = np.sqrt(MSE(test_y,pred))
print(rmse)

In [ ]:
pred_unit = xgb_r.predict(test_data[cols_list])

In [ ]:
final_pred = (pred_OHMM + pred_DEC + pred_GAPI + pred_USF + pred_IN + pred_)/6